# Quantum codes do not fix qubit independent errors
#### Authors: J. Lacalle, L. M. Pozo-Coronado, A. L. Fonseca de Oliveira, R. Martin-Cuevas

This notebook complements the homonymous article written by the authors, by proving several claims made in the article with the assistance of a symbolic calculus library for Python.

First of all, we will import the required libraries and set up a few basic variables and tools.

In [1]:
from sympy import symbols, Symbol, Matrix, I, pprint, diff, re, im, expand, degree
from sympy.core.expr import Expr
from sympy.physics.quantum import Dagger
from sympy.physics.quantum.tensorproduct import TensorProduct
from itertools import combinations

num_qubits = 5

def intersection(a: list, b: list) -> list:
    """
    Returns the intersection of two lists: a new list containing the elements
    that appeared in both input lists.
    
    :param a: First list to consider.
    :param b: Second list to consider.
    :result: Resulting list.
    """
    result = []
    
    for a_i in a:
        if a_i in b:
            result.append(a_i)
    
    return result

## 1. Finding the basis associated with the 5-qubit quantum code

We will start this section by defining the generators used by the 5-qubit quantum code $C$. As we will be using the string themselves to perform calculations on the quantum states, the Dirac notation is avoided when defining these variables.

In [2]:
L0 = Symbol('00000') - Symbol('00011') + Symbol('00101') - Symbol('00110') + \
     Symbol('01001') + Symbol('01010') - Symbol('01100') - Symbol('01111') - \
     Symbol('10001') + Symbol('10010') + Symbol('10100') - Symbol('10111') - \
     Symbol('11000') - Symbol('11011') - Symbol('11101') - Symbol('11110')

L1 = - Symbol('00001') - Symbol('00010') - Symbol('00100') - Symbol('00111') - \
     Symbol('01000') + Symbol('01011') + Symbol('01101') - Symbol('01110') - \
     Symbol('10000') - Symbol('10011') + Symbol('10101') + Symbol('10110') - \
     Symbol('11001') + Symbol('11010') - Symbol('11100') + Symbol('11111')

We will now define the Pauli matrices that will be used later on to construct the different discrete errors.

In [3]:
X = Matrix([[0, 1], [1, 0]])
Y = Matrix([[0, -I], [I, 0]])
Z = Matrix([[1, 0], [0, -1]])

The computational basis ($B_c$) can be defined as follows. We will also define a method to build any binary string, from the non-negative integer in decimal basis that it should represent.

In [4]:
def convert_to_binary_string(i: int, num_qubits: int) -> Symbol:
    """
    This method returns the given non-negative integer in decimal basis, expressed as
    a binary string with as many digits as required by the second parameter.

    :param i: Non-negative integer in decimal basis.
    :param num_bits: Length of the resulting binary string.
    :return: Sympy symbol containing the desired string as name.
    """
    binary_string = str(bin(i)[2:])
    binary_string = ''.join(['0' for _ in range(num_qubits - len(binary_string))]) + binary_string
    return Symbol(binary_string)


B_c = []  # Computational basis: [00000, 00001, ..., 11111].
for i in range(2 ** num_qubits):
    B_c.append(convert_to_binary_string(i, num_qubits))
print('B_c:', B_c)

B_c: [00000, 00001, 00010, 00011, 00100, 00101, 00110, 00111, 01000, 01001, 01010, 01011, 01100, 01101, 01110, 01111, 10000, 10001, 10010, 10011, 10100, 10101, 10110, 10111, 11000, 11001, 11010, 11011, 11100, 11101, 11110, 11111]


The basis associated with the quantum code $C$ ($B$) can be defined as follows, using the Pauli matrices, and aplying them on all the different qubits of each of the quantum states defined as generators, to obtain:
$B = [|0_L\rangle, |1_L\rangle, X_0|0_L\rangle, X_0|1_L\rangle, X_1|0_L\rangle, X_1|1_L\rangle, ..., Z_4|0_L\rangle, Z_4|1_L\rangle]$

We need to define an auxiliary method to apply 1-qubit operation to quantum states, using its string representation to perform the calculation. We will also need another method to obtain which new states could be obtained if we alter one bit of an incoming binary string.

In [5]:
def apply_1qubit_operation(operation: Matrix, target_qubit: int, initial_state: Expr, num_qubits: int) -> Expr:
    """
    This method applies the specified 1-qubit operation to the initial state, on the desired
    target qubit, and returns the resulting quantum state.
    
    :param operation: 2x2 matrix to be applied. 
    :param target_qubit: Qubit on which it should be applied, one of {0, 1, ..., num_qubits - 1}.
    :param initial_state: Initial quantum state, as an expression in terms of elements of the
        computational basis, expressed as binary strings.
    :param num_qubits: Number of qubits of the quantum state.
    :return: New quantum state after applying the operation.
    """
    
    final_state = 0

    for i in range(2 ** num_qubits):
        index = convert_to_binary_string(i, num_qubits)

        alpha = initial_state.coeff(index)
        state_0, state_1 = get_basis_states_one_bit_apart(index, target_qubit)

        initial_qubit_state = Matrix([[1], [0]]) if index.name[target_qubit] == '0' else Matrix([[0], [1]])
        final_qubit_state = operation * initial_qubit_state

        final_state += alpha * (final_qubit_state[0] * state_0 + final_qubit_state[1] * state_1)

    return final_state

def get_basis_states_one_bit_apart(reference_state: Symbol, target: int) -> tuple:
    """
    Method used to get the two binary strings that would be obtained if we modify any
    one bit of an incoming binary string. E.g.: if we receive the string 00000 and the
    integer '1' as target, the two possible outputs will be 00000 and 01000.
    
    :param reference_state: Incoming binary string. 
    :param target: Bit that would be altered, one of {0, 1, ..., length - 1}.
    :return: Tuple with the new two binary strings.
    """
    reference_state = reference_state.name

    return Symbol(reference_state[:target] + '0' + reference_state[target + 1:]), \
           Symbol(reference_state[:target] + '1' + reference_state[target + 1:])

B = [L0, L1]  # Basis associated with the code C.
for operator in [X, Y, Z]:
    for u in range(num_qubits):
        for generator in [L0, L1]:
            B.append(
                apply_1qubit_operation(operator, u, generator, num_qubits)
            )
print('B:', B)

B: [00000 - 00011 + 00101 - 00110 + 01001 + 01010 - 01100 - 01111 - 10001 + 10010 + 10100 - 10111 - 11000 - 11011 - 11101 - 11110, -00001 - 00010 - 00100 - 00111 - 01000 + 01011 + 01101 - 01110 - 10000 - 10011 + 10101 + 10110 - 11001 + 11010 - 11100 + 11111, -00001 + 00010 + 00100 - 00111 - 01000 - 01011 - 01101 - 01110 + 10000 - 10011 + 10101 - 10110 + 11001 + 11010 - 11100 - 11111, -00000 - 00011 + 00101 + 00110 - 01001 + 01010 - 01100 + 01111 - 10001 - 10010 - 10100 - 10111 - 11000 + 11011 + 11101 - 11110, 00001 + 00010 - 00100 - 00111 + 01000 - 01011 + 01101 - 01110 - 10000 - 10011 - 10101 - 10110 - 11001 + 11010 + 11100 - 11111, -00000 + 00011 + 00101 - 00110 - 01001 - 01010 - 01100 - 01111 - 10001 + 10010 - 10100 + 10111 - 11000 - 11011 + 11101 + 11110, 00001 - 00010 + 00100 - 00111 - 01000 - 01011 + 01101 + 01110 + 10000 - 10011 - 10101 + 10110 - 11001 - 11010 - 11100 - 11111, -00000 - 00011 - 00101 - 00110 + 01001 - 01010 - 01100 + 01111 + 10001 + 10010 - 10100 - 10111 - 11000 

## 2. Finding the change of basis matrices

Now we will define the matrix $M$ used to transform from basis $B$ to basis $B_c$. We will define it in two ways, firstly by putting the vectors from $B$, expressed in tearms of $B_c$, as columns. Then, we will define it by using derivatives. After that, we will ensure that we can get all vectors of the base $B$ expressed in the computational basis $B_c$ and that matches the expression of the basis $B$ obtained earlier, and also that both matrices $M$ are equal, as confirmations.

In [6]:
M = Matrix([
    [
        B[column].coeff(convert_to_binary_string(row, num_qubits))
        for column in range(2 ** num_qubits)
    ] for row in range(2 ** num_qubits)
])

In [7]:
M2 = Matrix([
    [
        diff(B[j], B_c[i]) for j in range(2 ** num_qubits)
    ] for i in range(2 ** num_qubits)
])

In [8]:
def get_computational_basis_vector_state(i: int, num_qubits: int) -> Matrix:
    """
    This methods returns a column vector that expresses the quantum state from the
    computational basis expressed by the integer 'i', for a quantum state with a total
    of 'num_qubits' qubits.

    :param i: Non-negative integer in decimal basis.
    :param num_qubits: Number of qubits of the resulting quantum state.
    :return: Sympy matrix containing the quantum state as a column vector.
    """
    return Matrix([
        [
            1 if i == j else 0
        ] for j in range(2 ** num_qubits)
    ])

def transform_vector_state_to_expression_state(state: Matrix, num_qubits: int) -> Expr:
    """
    This method converts a column vector to an expression in terms of the states of
    the corresponding basis, as binary strings, to be used in methods that use these
    binary strings to perform calculations.
    
    :param state: Matrix containing a quantum state expressed as a column vector.
    :param num_qubits: Number of qubits of the quantum state.
    :return: Quantum state, as expression with its variables as binary strings.
    """

    result = 0

    for j in range(state.shape[0]):
        binary_string = convert_to_binary_string(j, num_qubits)
        result += state[j, 0] * binary_string

    return result

assert 0 == sum(M - M2)
for i in range(2 ** num_qubits):
    state_B_vector = M * get_computational_basis_vector_state(i, num_qubits)
    state_B_c_expression = transform_vector_state_to_expression_state(state_B_vector, num_qubits)
    assert state_B_c_expression == B[i]  # The program will fail if the assertion is false.
print('Successfully proven that M transforms vectors from B to B_c.')

print('Shape of M == M2:', M.shape)
#pprint(M)

Successfully proven that M transforms vectors from B to B_c.
Shape of M == M2: (32, 32)


## 3. Finding the unitary error operators

Now we will define the qubit errors $W_u$ in terms of the coefficients $A_u$, $B_u$, $C_u$ and $D_u$.

In [9]:
A = symbols('A_0:%d' % num_qubits)  # Define A_0 through A_4.
B = symbols('B_0:%d' % num_qubits)  # Define B_0 through B_4.
C = symbols('C_0:%d' % num_qubits)  # Define C_0 through C_4.
D = symbols('D_0:%d' % num_qubits)  # Define D_0 through D_4.
V = A + B + C + D

W_u = []
for u in range(num_qubits):  # Define W_0 through W_4.
    W_u.append(Matrix([
        [A[u] + I * B[u], -C[u] + I * D[u]],
        [C[u] + I * D[u], A[u] - I * B[u]]
    ]))
    print('W[%d]:' % u, W_u[u])

W[0]: Matrix([[A_0 + I*B_0, -C_0 + I*D_0], [C_0 + I*D_0, A_0 - I*B_0]])
W[1]: Matrix([[A_1 + I*B_1, -C_1 + I*D_1], [C_1 + I*D_1, A_1 - I*B_1]])
W[2]: Matrix([[A_2 + I*B_2, -C_2 + I*D_2], [C_2 + I*D_2, A_2 - I*B_2]])
W[3]: Matrix([[A_3 + I*B_3, -C_3 + I*D_3], [C_3 + I*D_3, A_3 - I*B_3]])
W[4]: Matrix([[A_4 + I*B_4, -C_4 + I*D_4], [C_4 + I*D_4, A_4 - I*B_4]])


With this, we can express the unitary error operator for independent qubit errors $W$, and that same operator in the base $B$, $W_B$:

In [10]:
W = TensorProduct(W_u[0], W_u[1], W_u[2], W_u[3], W_u[4])
print('Shape of W:', W.shape)
#pprint(W)

Shape of W: (32, 32)


In [11]:
W_B = Dagger(M) * W * M
print('Shape of W_B:', W.shape)
print('W[0][0] =', expand(W_B[0, 0]))

Shape of W_B: (32, 32)
W[0][0] = 16*A_0*A_1*A_2*A_3*A_4 + 16*I*A_0*A_1*B_3*C_2*C_4 + 16*I*A_0*A_2*B_1*D_3*D_4 + 16*I*A_0*A_3*B_4*D_1*D_2 + 16*I*A_0*A_4*B_2*C_1*C_3 + 16*A_0*B_1*B_4*C_2*C_3 + 16*A_0*B_2*B_3*D_1*D_4 + 16*A_0*C_1*C_4*D_2*D_3 + 16*I*A_1*A_2*B_4*C_0*C_3 + 16*I*A_1*A_3*B_2*D_0*D_4 + 16*I*A_1*A_4*B_0*D_2*D_3 + 16*A_1*B_0*B_2*C_3*C_4 + 16*A_1*B_3*B_4*D_0*D_2 + 16*A_1*C_0*C_2*D_3*D_4 + 16*I*A_2*A_3*B_0*C_1*C_4 + 16*I*A_2*A_4*B_3*D_0*D_1 + 16*A_2*B_0*B_4*D_1*D_3 + 16*A_2*B_1*B_3*C_0*C_4 + 16*A_2*C_1*C_3*D_0*D_4 + 16*I*A_3*A_4*B_1*C_0*C_2 + 16*A_3*B_0*B_1*D_2*D_4 + 16*A_3*B_2*B_4*C_0*C_1 + 16*A_3*C_2*C_4*D_0*D_1 + 16*A_4*B_0*B_3*C_1*C_2 + 16*A_4*B_1*B_2*D_0*D_3 + 16*A_4*C_0*C_3*D_1*D_2 + 16*I*B_0*B_1*B_2*B_3*B_4 + 16*I*B_0*C_2*C_3*D_1*D_4 + 16*I*B_1*C_3*C_4*D_0*D_2 + 16*I*B_2*C_0*C_4*D_1*D_3 + 16*I*B_3*C_0*C_1*D_2*D_4 + 16*I*B_4*C_1*C_2*D_0*D_3


As discussed in the article, each of the entries in matrix $W_B$ is a homogeneous polynomial of degree 5 in the set of variables $V = \{A_u, B_u, C_u, D_u | 0 \le u \lt 5\}$, and degree 1 in the subsets of variables of $V$ with subscript $u$ for all $0 \le u \lt 5$. Each of this polynomials includes exactly 32 monomials.

## 4. Disturbing a quantum state

We can define a generic quantum state in basis $B$, $\phi_B$ (phi_B), as follows:

In [12]:
w = symbols('w_0:%d' % 4)  # Define w_0 through w_3.

phi_B = Matrix([
    [w[0] + I * w[1]],
    [w[2] + I * w[3]]
] + [
    [0]
    for _ in range(2 ** num_qubits - 2)
])
print('Shape of phi_B:', phi_B.shape)
print(phi_B)

Shape of phi_B: (32, 1)
Matrix([[w_0 + I*w_1], [w_2 + I*w_3], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]])


The disturbed quantum state $\psi_B$ (psi_B) is obtained by multiplying $\phi_B$ by the error operator $W_B$:

In [13]:
psi_B = W_B * phi_B
print('Shape of psi_B:', psi_B.shape)
print('psi_B[0] = beta_0 =', expand(psi_B[0, 0]))

Shape of psi_B: (32, 1)
psi_B[0] = beta_0 = 16*A_0*A_1*A_2*A_3*A_4*w_0 + 16*I*A_0*A_1*A_2*A_3*A_4*w_1 + 16*I*A_0*A_1*B_2*B_4*D_3*w_2 - 16*A_0*A_1*B_2*B_4*D_3*w_3 + 16*I*A_0*A_1*B_3*C_2*C_4*w_0 - 16*A_0*A_1*B_3*C_2*C_4*w_1 - 16*A_0*A_1*C_3*D_2*D_4*w_2 - 16*I*A_0*A_1*C_3*D_2*D_4*w_3 + 16*I*A_0*A_2*B_1*D_3*D_4*w_0 - 16*A_0*A_2*B_1*D_3*D_4*w_1 - 16*A_0*A_2*B_3*B_4*C_1*w_2 - 16*I*A_0*A_2*B_3*B_4*C_1*w_3 + 16*I*A_0*A_2*C_3*C_4*D_1*w_2 - 16*A_0*A_2*C_3*C_4*D_1*w_3 - 16*A_0*A_3*B_1*B_2*C_4*w_2 - 16*I*A_0*A_3*B_1*B_2*C_4*w_3 + 16*I*A_0*A_3*B_4*D_1*D_2*w_0 - 16*A_0*A_3*B_4*D_1*D_2*w_1 + 16*I*A_0*A_3*C_1*C_2*D_4*w_2 - 16*A_0*A_3*C_1*C_2*D_4*w_3 + 16*I*A_0*A_4*B_1*B_3*D_2*w_2 - 16*A_0*A_4*B_1*B_3*D_2*w_3 + 16*I*A_0*A_4*B_2*C_1*C_3*w_0 - 16*A_0*A_4*B_2*C_1*C_3*w_1 - 16*A_0*A_4*C_2*D_1*D_3*w_2 - 16*I*A_0*A_4*C_2*D_1*D_3*w_3 + 16*A_0*B_1*B_4*C_2*C_3*w_0 + 16*I*A_0*B_1*B_4*C_2*C_3*w_1 + 16*A_0*B_2*B_3*D_1*D_4*w_0 + 16*I*A_0*B_2*B_3*D_1*D_4*w_1 + 16*A_0*C_1*C_4*D_2*D_3*w_0 + 16*I*A_0*C_1*C_4*D_2*D_3*w_

The coordinates of $\psi_B$ are homogeneous polynomials of degree 5 in the set of variables $V$, degree 1 in the subsets of variables of $V$ with subscript $u$ for all $0 \le u \lt 5$, and degree 1 in the variables $w_0$, $w_1$, $w_2$ and $w_3$. It can be expressed as $\psi_B = (\beta_0, \beta_1, ..., \beta_{31})_B$.

### 4.1 Proving Lemma 1

Lemma 1 states that the coordinates of the disturbed state $\psi_B = (\beta_0, \beta_1, ..., \beta_{31})_B$ satisfy:

$$ \beta_{2s} = (a_s + ib_s)w_0 + (-b_s + ia_s)w_1 + (c_s + id_s)w_2 + (-d_s + ic_s)w_3, 0 \le s \lt 16 $$
$$ \beta_{2s+1} = (-c_s + id_s)w_0 + (-d_s - ic_s)w_1 + (a_s - ib_s)w_2 + (b_s + ia_s)w_3, s = 0 $$
$$ \beta_{2s+1} = (c_s - id_s)w_0 + (d_s + ic_s)w_1 + (-a_s + ib_s)w_2 + (-b_s - ia_s)w_3, 0 \lt s \lt 16  $$

where

$$ a_s = Re(\beta_{2s}(1,0,0,0)), b_s = Im(\beta_{2s}(1,0,0,0)), $$
$$ c_s = Re(\beta_{2s}(0,0,1,0)), d_s = Im(\beta_{2s}(0,0,1,0)), $$
$$ 0 \le s \lt 16 $$

In [14]:
def separate_real_from_imaginary(expression: Expr) -> tuple:

    polynomial = expand(expression)
    monomials = expand(expression).args
    
    real = polynomial  # We will substract the imaginary terms from this polynomial.
    imaginary = 0      # We will add the imaginary terms from this polynomial.
    
    for monomial in monomials:

        i = 0
        monomial_has_imaginary_unit = False
        variables = monomial.args
        while i < len(variables) and not monomial_has_imaginary_unit:
            if variables[i] == I:
                monomial_has_imaginary_unit = True
            i += 1

        if monomial_has_imaginary_unit:
            real -= monomial
            imaginary += monomial

    return expand(real), expand(imaginary / I)


def get_beta_2s(s: int, a_s: Expr, b_s: Expr, c_s: Expr, d_s: Expr, w: list) -> Expr:
    
    return expand(
        (a_s + I*b_s) * w[0] + (-b_s + I*a_s) * w[1] +
        (c_s + I*d_s) * w[2] + (-d_s + I*c_s) * w[3]
    )


def get_beta_2s_1(s: int, a_s: Expr, b_s: Expr, c_s: Expr, d_s: Expr, w: list) -> Expr:
    
    if s == 0:
        return expand(  # When s = 0.
            (-c_s + I*d_s) * w[0] + (-d_s - I*c_s) * w[1] +
            (a_s - I*b_s) * w[2] + (b_s + I*a_s) * w[3]
        )
    else:
        return expand(  # When 0 < s < 16.
            (c_s - I*d_s) * w[0] + (d_s + I*c_s) * w[1] +
            (-a_s + I*b_s) * w[2] + (-b_s - I*a_s) * w[3]
        )

a = []
b = []
c = []
d = []
for s in range(16):

    beta_2s_1000 = psi_B[2 * s].subs(w[0], 1).subs(w[1], 0).subs(w[2], 0).subs(w[3], 0)
    real, imaginary = separate_real_from_imaginary(beta_2s_1000)
    a.append(real)       # Save a_s.
    b.append(imaginary)  # Save b_s.

    beta_2s_0010 = psi_B[2 * s].subs(w[0], 0).subs(w[1], 0).subs(w[2], 1).subs(w[3], 0)
    real, imaginary = separate_real_from_imaginary(beta_2s_0010)
    c.append(real)       # Save c_s.
    d.append(imaginary)  # Save d_s.

    # Test beta_2s.
    beta_2s = get_beta_2s(s, a[s], b[s], c[s], d[s], w)
    psi_B_2s = expand(psi_B[2 * s, 0])
    assert beta_2s == psi_B_2s  # The program will fail if the assertion is false.
    print('Lemma 1 successfully proven for beta_2s with s = %d' % s)

    # Test beta_2s+1.
    beta_2s_1 = get_beta_2s_1(s, a[s], b[s], c[s], d[s], w)
    psi_B_2s_1 = expand(psi_B[2 * s + 1, 0])
    assert beta_2s_1 == psi_B_2s_1  # The program will fail if the assertion is false.
    print('Lemma 1 successfully proven for beta_2s+1 with s = %d' % s)

Lemma 1 successfully proven for beta_2s with s = 0
Lemma 1 successfully proven for beta_2s+1 with s = 0
Lemma 1 successfully proven for beta_2s with s = 1
Lemma 1 successfully proven for beta_2s+1 with s = 1
Lemma 1 successfully proven for beta_2s with s = 2
Lemma 1 successfully proven for beta_2s+1 with s = 2
Lemma 1 successfully proven for beta_2s with s = 3
Lemma 1 successfully proven for beta_2s+1 with s = 3
Lemma 1 successfully proven for beta_2s with s = 4
Lemma 1 successfully proven for beta_2s+1 with s = 4
Lemma 1 successfully proven for beta_2s with s = 5
Lemma 1 successfully proven for beta_2s+1 with s = 5
Lemma 1 successfully proven for beta_2s with s = 6
Lemma 1 successfully proven for beta_2s+1 with s = 6
Lemma 1 successfully proven for beta_2s with s = 7
Lemma 1 successfully proven for beta_2s+1 with s = 7
Lemma 1 successfully proven for beta_2s with s = 8
Lemma 1 successfully proven for beta_2s+1 with s = 8
Lemma 1 successfully proven for beta_2s with s = 9
Lemma 1 succe

### 4.2 Proving Lemma 3

Lemma 3 states that

$$ a_s = Re(\beta_{2s}(1,0,0,0)), b_s = Im(\beta_{2s}(1,0,0,0)), $$
$$ c_s = Re(\beta_{2s}(0,0,1,0)), d_s = Im(\beta_{2s}(0,0,1,0)), $$
$$ 0 \le s \lt 16 $$

Satisfy the following properties:

1) They are homogeneous polynomials of degree 5 in the set of variables $V = \{A_u, B_u, C_u, D_u | 0 \le u \lt 5\}$, degree 1 in the subsets of variables of $V$, and they are made up of 16 monomials.

In [15]:
for x, name in [(a, 'a'), (b, 'b'), (c, 'c'), (d, 'd')]:
    for s in range(16):
        
        monomials = expand(x[s]).args
        
        # Polynomials made up of 16 monomials.
        assert len(monomials) == 16
        
        for monomial in monomials:
            
            monomial_degree = 0
            variables = monomial.args
            
            for variable in variables:
                
                if variable in V:
                    monomial_degree += 1
                    
                    # Degree 1 in the subsets of variables of 𝑉.
                    assert degree(monomial, gen=variable) == 1
            
            # Degree 5 in the set 
            assert monomial_degree == 5
    
    print('Lemma 3, first property, successfully proven for %s[0] to %s[%d]' % (name, name, s))

Lemma 3, first property, successfully proven for a[0] to a[15]
Lemma 3, first property, successfully proven for b[0] to b[15]
Lemma 3, first property, successfully proven for c[0] to c[15]
Lemma 3, first property, successfully proven for d[0] to d[15]


2) The monomials of $a_s$, $b_s$, $c_s$ and $d_s$ for all $0 \le s \lt 16$ are different two by two.

In [16]:
all_monomials = {}  # Dictionary used as hash table to keep track of monomials seen.

for x, name in [(a, 'a'), (b, 'b'), (c, 'c'), (d, 'd')]:
    for s in range(16):
        
        monomials = expand(x[s]).args
        
        for monomial in monomials:
            
            monomial = str(monomial)
            assert monomial not in all_monomials.keys()
            all_monomials[monomial] = True
    
    print('Lemma 3, second property, successfully proven for %s[0] to %s[%d]' % (name, name, s))

Lemma 3, second property, successfully proven for a[0] to a[15]
Lemma 3, second property, successfully proven for b[0] to b[15]
Lemma 3, second property, successfully proven for c[0] to c[15]
Lemma 3, second property, successfully proven for d[0] to d[15]


3) For all $0 \le s \lt 16$, in each of $a_s$, $b_s$, $c_s$ or $d_s$, every pair of different monomials share exactly one variable.

In [17]:
for x, name in [(a, 'a'), (b, 'b'), (c, 'c'), (d, 'd')]:
    for s in range(16):
        
        monomials = expand(x[s]).args
        
        i = 0
        while i < len(monomials):
            monomial_1 = monomials[i]
            variables_monomial_1 = monomial_1.args
            
            j = i + 1
            while j < len(monomials):
                monomial_2 = monomials[j]
                variables_monomial_2 = monomial_2.args
                
                shared_variables = 0
                for variable in variables_monomial_2:
                    if variable in V and variable in variables_monomial_1:
                        shared_variables += 1
                
                # Every pair of different monomials within the same polynomial share exactly one variable.
                assert shared_variables == 1
                
                j += 1            
            i += 1
    
    print('Lemma 3, third property, successfully proven for %s[0] to %s[%d]' % (name, name, s))

Lemma 3, third property, successfully proven for a[0] to a[15]
Lemma 3, third property, successfully proven for b[0] to b[15]
Lemma 3, third property, successfully proven for c[0] to c[15]
Lemma 3, third property, successfully proven for d[0] to d[15]


## 5. Classification of monomials by type

In [18]:
def count_As_in_monomial(monomial: Expr, A: list) -> int:
    
    result = 0
    
    for variable in monomial.args:
        if variable in A:
            result += 1
            
    return result


for s in range(16):
    
    # Create the list that will keep track of how many instances of each type of monomial are there.
    # E.g.: if type_counter[5] == 1, that means that there is one monomial with 5 A's.
    # The array goes from counter[0] (0 A's), to counter[5] (5 A's).
    type_counter = [0 for _ in range(len(A) + 1)]

    for x, name in [(a, 'a'), (b, 'b'), (c, 'c'), (d, 'd')]:

        monomials = expand(x[s]).args

        for monomial in monomials:

            num_As = count_As_in_monomial(monomial, A)
            type_counter[num_As] += 1

    if s == 0:
        assert type_counter == [18, 15, 30, 0, 0, 1]
    elif 0 < s < 16:
        assert type_counter == [15, 26, 16, 6, 1, 0]
    print('Monomial type count proven for s = %d' % (s))

Monomial type count proven for s = 0
Monomial type count proven for s = 1
Monomial type count proven for s = 2
Monomial type count proven for s = 3
Monomial type count proven for s = 4
Monomial type count proven for s = 5
Monomial type count proven for s = 6
Monomial type count proven for s = 7
Monomial type count proven for s = 8
Monomial type count proven for s = 9
Monomial type count proven for s = 10
Monomial type count proven for s = 11
Monomial type count proven for s = 12
Monomial type count proven for s = 13
Monomial type count proven for s = 14
Monomial type count proven for s = 15


## 6. Search of subsets of variables $V_s$

The reader may find an algorithm for the identification of subsets $V_s$ in the file "aux_functions.py", included in this very repository. The following code imports and runs the algorithm from that file.

In [19]:
import aux_functions
import importlib
importlib.reload(aux_functions)


# If this parameter is marked as False, for each 0 ≤ s < 16, the program will look
# for just one valid value of V_s.
get_all_combinations = False

# If this parameter is marked as True, for 0 < s < 16, the program will look for
# solutions that do not include any variable A_u, for all 0 ≤ u < 5.
avoid_As_where_possible = True

# Run:
for s in range(0, 16):
    
    if avoid_As_where_possible and s > 0:
        print('\ns =', s, '(avoiding any variable A_u, for all 0 ≤ u < 5)')
        valid_variables = B + C + D
    else:
        print('\ns =', s)
        valid_variables = A + B + C + D
    
    result = []
    monomials = aux_functions.get_all_monomials(a[s], b[s], c[s], d[s], valid_variables)
    
    aux_functions.search_configurations_recursively(s, {}, monomials, result, get_all_combinations)
    print('  ', len(result), 'possible V_' + str(s) + '(\'s) found.')


s = 0
   Valid V_0 found: ['A_0', 'A_1', 'A_2', 'B_1', 'B_2', 'B_3', 'C_3', 'D_0']
   1 possible V_0('s) found.

s = 1 (avoiding any variable A_u, for all 0 ≤ u < 5)
   Valid V_1 found: ['B_0', 'B_1', 'B_2', 'B_3', 'C_1', 'C_2', 'D_0', 'D_3']
   1 possible V_1('s) found.

s = 2 (avoiding any variable A_u, for all 0 ≤ u < 5)
   Valid V_2 found: ['B_1', 'B_2', 'B_3', 'B_4', 'C_2', 'C_3', 'D_1', 'D_4']
   1 possible V_2('s) found.

s = 3 (avoiding any variable A_u, for all 0 ≤ u < 5)
   Valid V_3 found: ['B_0', 'B_1', 'B_2', 'B_4', 'C_0', 'C_1', 'D_2', 'D_4']
   1 possible V_3('s) found.

s = 4 (avoiding any variable A_u, for all 0 ≤ u < 5)
   Valid V_4 found: ['B_0', 'B_1', 'B_3', 'B_4', 'C_0', 'C_4', 'D_1', 'D_3']
   1 possible V_4('s) found.

s = 5 (avoiding any variable A_u, for all 0 ≤ u < 5)
   Valid V_5 found: ['B_0', 'B_1', 'B_2', 'B_4', 'C_0', 'C_1', 'D_2', 'D_4']
   1 possible V_5('s) found.

s = 6 (avoiding any variable A_u, for all 0 ≤ u < 5)
   Valid V_6 found: ['B_0', 'B_2'

We will test a few sets of length 8 obtained during the development of the article, to ensure they are valid.

In [20]:
V_1  = [B[3], B[4], C[0], C[2], D[0], D[2], D[3], D[4]]
V_2  = [B[2], B[3], C[1], C[4], D[1], D[2], D[3], D[4]]
V_3  = [B[2], B[3], C[1], C[4], D[1], D[2], D[3], D[4]]
V_4  = [B[2], B[3], C[1], C[4], D[1], D[2], D[3], D[4]]
V_5  = [B[2], B[3], C[1], C[4], D[1], D[2], D[3], D[4]]
V_6  = [B[3], B[4], C[0], C[2], D[0], D[2], D[3], D[4]]
V_7  = [B[2], B[3], C[1], C[4], D[1], D[2], D[3], D[4]]
V_8  = [B[3], B[4], C[0], C[2], D[0], D[2], D[3], D[4]]
V_9  = [B[0], B[4], C[1], C[3], D[0], D[1], D[3], D[4]]
V_10 = [B[2], B[3], C[1], C[4], D[1], D[2], D[3], D[4]]
V_11 = [B[0], B[4], C[1], C[3], D[0], D[1], D[3], D[4]]
V_12 = [B[0], B[1], C[2], C[4], D[0], D[1], D[2], D[4]]
V_13 = [B[2], B[3], C[1], C[4], D[1], D[2], D[3], D[4]]
V_14 = [B[2], B[3], C[1], C[4], D[1], D[2], D[3], D[4]]
V_15 = [B[3], B[4], C[0], C[2], D[0], D[2], D[3], D[4]]
valid_variables = A + B + C + D

sets_to_test = [V_1, V_2, V_3, V_4, V_5, V_6, V_7, V_8, V_9, V_10, V_11, V_12, V_13, V_14, V_15]
set_index = 0
while set_index < len(sets_to_test):
    set_under_test = sets_to_test[set_index]
    
    s = set_index + 1
    for polynomial in [a[s], b[s], c[s], d[s]]:
        for monomial in expand(polynomial).args:
            variables = [var for var in monomial.args if var in valid_variables]
            
            # The intersection of each of the previous sets with the set of variables of
            # any monomial of a_0, b_0, c_0 or d_0 has an odd cardinal.
            assert len(intersection(set_under_test, variables)) % 2 == 1
        
    print('Assertion tested for V_' + str(s))
    set_index += 1

Assertion tested for V_1
Assertion tested for V_2
Assertion tested for V_3
Assertion tested for V_4
Assertion tested for V_5
Assertion tested for V_6
Assertion tested for V_7
Assertion tested for V_8
Assertion tested for V_9
Assertion tested for V_10
Assertion tested for V_11
Assertion tested for V_12
Assertion tested for V_13
Assertion tested for V_14
Assertion tested for V_15


## 7. Checking the intersection of $V'_0$, $V'_1$, $V'_2$, $V'_3$ and $V'_4$ with different monomials

Some valid definitions of $V_s$ will now be used to test several properties.

In [21]:
V_prime_0 = [A[0], B[0], B[3], B[4], C[2], D[2], D[3], D[4]]
V_prime_1 = [A[1], B[1], B[2], B[3], C[4], D[2], D[3], D[4]]
V_prime_2 = [A[2], B[0], C[0], C[3], C[4], D[2], D[3], D[4]]
V_prime_3 = [A[3], B[1], B[2], B[4], C[2], D[1], D[3], D[4]]
V_prime_4 = [A[4], B[1], C[1], C[2], C[3], D[2], D[3], D[4]]
valid_variables = A + B + C + D

sets_to_test = [V_prime_0, V_prime_1, V_prime_2, V_prime_3, V_prime_4]
set_index = 0
while set_index < len(sets_to_test):
    set_under_test = sets_to_test[set_index]
    
    for polynomial in [a[0], b[0], c[0], d[0]]:
        for monomial in expand(polynomial).args:
            variables = [var for var in monomial.args if var in valid_variables]
            
            # The intersection of each of the previous sets with the set of variables of
            # any monomial of a_0, b_0, c_0 or d_0 has an odd cardinal.
            assert len(intersection(set_under_test, variables)) % 2 == 1
    
    # The intersection of each of the previous sets with the set of variables of the
    # monomial A_0*A_1*A_2*A_3*A_4 is {A_0}, {A_1}, {A_2}, {A_3} and {A_4} respectively.
    assert intersection(set_under_test, A) == [A[set_index]]
        
    print('Assertion tested for V_prime_' + str(set_index))
    set_index += 1

Assertion tested for V_prime_0
Assertion tested for V_prime_1
Assertion tested for V_prime_2
Assertion tested for V_prime_3
Assertion tested for V_prime_4


## 8. Checking the intersection of $V''$ with different monomials

In [22]:
V_doubleprime_2  = [A[1], B[1], B[2], B[3], C[4], D[2], D[3], D[4]]
V_doubleprime_3  = [A[1], B[2], C[0], C[1], C[3], D[0], D[2], D[3]]
V_doubleprime_4  = [A[1], B[2], B[3], B[4], C[1], C[2], C[3], D[4]]
V_doubleprime_5  = [A[0], B[0], B[3], B[4], C[2], D[2], D[3], D[4]]
V_doubleprime_6  = [A[0], B[0], B[1], B[3], C[1], C[3], C[4], D[4]]
V_doubleprime_7  = [A[0], B[2], B[3], B[4], C[0], C[3], C[4], D[2]]
V_doubleprime_8  = [A[0], B[0], B[1], B[3], C[1], C[3], C[4], D[4]]
V_doubleprime_9  = [A[0], B[1], B[2], B[4], C[1], D[0], D[2], D[4]]
V_doubleprime_10 = [A[0], B[1], B[2], B[3], C[0], C[1], C[2], D[3]]
V_doubleprime_11 = [A[0], B[0], B[2], B[4], C[1], C[2], C[4], D[1]]
V_doubleprime_12 = [A[0], B[2], C[2], C[3], C[4], D[0], D[3], D[4]]
V_doubleprime_13 = [A[0], B[0], B[3], B[4], C[2], D[2], D[3], D[4]]
V_doubleprime_14 = [A[0], B[0], B[1], B[2], C[3], D[1], D[2], D[3]]
V_doubleprime_15 = [A[0], B[1], C[0], C[2], C[4], D[1], D[2], D[4]]
V_doubleprime_16 = [A[0], B[1], B[2], B[3], C[0], C[1], C[2], D[3]]
valid_variables = A + B + C + D

sets_to_test = [V_doubleprime_2, V_doubleprime_3, V_doubleprime_4, V_doubleprime_5, V_doubleprime_6,
                V_doubleprime_7, V_doubleprime_8, V_doubleprime_9, V_doubleprime_10, V_doubleprime_11,
                V_doubleprime_12, V_doubleprime_13, V_doubleprime_14, V_doubleprime_15, V_doubleprime_16]

polynomials = [expand(polynomial) for polynomial in [a[0], b[0], c[0], d[0]]]
monomials_from_a0 = expand(a[0]).args

set_index = 0
while set_index < len(sets_to_test):
    set_under_test = sets_to_test[set_index]
    
    for polynomial in polynomials:
        for monomial in polynomial.args:
            variables = [var for var in monomial.args if var in valid_variables]
            
            # The intersection of each of the previous sets with the set of variables of
            # any monomial of a_0, b_0, c_0 or d_0 has an odd cardinal.
            assert len(intersection(set_under_test, variables)) % 2 == 1
    
    # The intersection of each of the previous sets with the corresponding monomial from
    # a_0 equals the second variable from the monomial (the first one different from A_u).
    target_monomial_object = monomials_from_a0[set_index + 1].args
    target_monomial_list = [var for var in target_monomial_object if var in valid_variables]
    expected_variable = target_monomial_list[1]
    assert intersection(set_under_test, target_monomial_list) == [expected_variable]
        
    print('Assertion tested for V_doubleprime_' + str(set_index + 2))
    set_index += 1

Assertion tested for V_doubleprime_2
Assertion tested for V_doubleprime_3
Assertion tested for V_doubleprime_4
Assertion tested for V_doubleprime_5
Assertion tested for V_doubleprime_6
Assertion tested for V_doubleprime_7
Assertion tested for V_doubleprime_8
Assertion tested for V_doubleprime_9
Assertion tested for V_doubleprime_10
Assertion tested for V_doubleprime_11
Assertion tested for V_doubleprime_12
Assertion tested for V_doubleprime_13
Assertion tested for V_doubleprime_14
Assertion tested for V_doubleprime_15
Assertion tested for V_doubleprime_16
